<a href="https://colab.research.google.com/github/maryantonopoulou/teliki_ergasia_EDDE2/blob/main/naftemborikiarticles_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site ώστε να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

from tqdm import tqdm # για να βάλω progress bar με το οποίο θα παρακολουθώ την πορεία του scraping

In [ ]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    #print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_article_elements_from_soup(soup, article_url):
  # απομόνωση του block με το περιεχόμενο του άρθρου απόο όλη τη σούπα της σελίδας
  article_block = soup.find("article")
  # article url
  article_url = article_url # το παίρνω από το arguement του function
  # title
  try:
    title = soup.find("h1" , class_=lambda c: c and "title" in c).text.strip()
  except:
    title=""
  # datetime
  try:
    datetime_str = soup.find("time")["datetime"]
    date_str = datetime_str.split("T")[0]
    time_str = datetime_str.split("T")[1].split("+")[0]
  except:
    date_str = ""
    time_str = ""
  # section
  try:
    section = soup.find("div", class_="post-category blue").text.strip().replace("\n", "")
  except:
    section = ""
  # author
  try:
    author = soup.find("div", class_="post-author").text.strip()
  except:
    author = ""
  # excerpt
  try:
   excerpt = soup.find("div", class_="post-content mb-25").find("h3").text.strip()
   excerpt = excerpt.replace("\xa0", "").strip()
  except:
    excerpt=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", class_= "post-content mb-25").find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
    text_body = " ".join(text_body_textsL)
  except:
    text_body=""
  # feature image
  try:
    feat_img_link = soup.find("img", src=True)["src"]
  except:
    feat_img_link=""
  # feature image caption
  try:
    image_caption = soup.find("div", class_="entry-summary").find("p").text.strip()
  except:
    image_caption=""

  # Δημιουργία λεξικού με τα data του άρθρου
  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excerpt": excerpt, "text_body": text_body, "feat_img_link": feat_img_link, "feat_img_cation": image_caption}

  return articleD

In [ ]:
def scrape_naftemporiki_articles_pages(teasers_df):
  # δημιουργία κενής λίστας στην οποία θα προσθέτουμε ένα-ένα τα λεξικά με τα δεδομένα των άρθρων που θα σκραπάρουμε
  all_articles_dataL = []
  # λούπα σε κάθε row του teasers_df για να παίρνουμε ένα-ένα τα urls των άρθρων και να το σκραπάρουμε
  # χρησιμοποιούμε το tqdm για να βλέπουμε σε ένα progress bar πού βρισκόμαστε και πόσος χρόνος απομένει ακόμα
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    # παίρνουμε το url του άρθρου που θα σκραπάρουμε από τη στήλη "article_url" του row του teasers_df στο οποίο βρίσκεται η for loop
    article_url = row['article_url']
    # scrape της σελίδας του url του κάθε άρθρου με το function που φτιάξαμε στο βήμα 2
    soup = get_webpage_soup(article_url)
    # δημιουργία του λεξικού με τα δεδομένα του κάθε άρθρου με το function που φτιάξαμε στο βήμα 9
    articleD = get_article_elements_from_soup(soup, article_url)
    # προσθήκη του λεξικού με τα δεδομένα του κάθε άρθρου στη λίστα που δημιουργήσαμε στη αρχή
    all_articles_dataL.append(articleD)
    # καθυστέρηση ανάμεσα στα requests στο server
    delay = random.uniform(1, 3)
    time.sleep(delay)
  # δημιουργία dataframe από τη λίστα των λεξικών με τα δεδομένα κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
teasers_df = pd.read_csv("/content/drive/MyDrive/CSVFILES/naftemporiki_teasers.csv")
teasers_df.head()

,title,article_url,date,time,section
0,Θεσσαλονίκη: Σοβαρό εργατικό ατύχημα στη Βιομη...,https://www.naftemporiki.gr/society/1964633/th...,04/06/2025,12:27,Κοινωνία
1,Θεσσαλονίκη: Σοβαρό εργατικό ατύχημα – Αυτοκίν...,https://www.naftemporiki.gr/society/1952668/th...,06/05/2025,07:47,Κοινωνία
2,Ηράκλειο: Εργατικό ατύχημα σε επιχείρηση με μά...,https://www.naftemporiki.gr/society/1934740/ir...,19/03/2025,10:44,Κοινωνία
3,Κοζάνη: Εργατικό ατύχημα με 22χρονο σοβαρά τρα...,https://www.naftemporiki.gr/society/1806579/ko...,24/10/2024,18:01,Κοινωνία
4,ΠΟΘΑ: Στάση εργασίας και συγκέντρωση διαμαρτυρ...,https://www.naftemporiki.gr/culture/ekdiloseis...,16/10/2024,13:08,Εκδηλώσεις


In [ ]:
articles_df = scrape_naftemporiki_articles_pages(teasers_df)

Processing rows: 100%|██████████| 35/35 [02:06<00:00,  3.60s/it]


Scraping completed. Fetshed 35 articles.


In [ ]:
articles_df

,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.naftemporiki.gr/society/1964633/th...,Θεσσαλονίκη: Σοβαρό εργατικό ατύχημα στη Βιομη...,2025-06-04,12:27:47,Κοινωνία,,,Σοβαρό εργατικό ατύχημα σημειώθηκε το πρωί της...,https://www.naftemporiki.gr/wp-content/themes/...,Σε σοβαρή κατάσταση ένας 34χρονος άνδρας
1,https://www.naftemporiki.gr/society/1952668/th...,Θεσσαλονίκη: Σοβαρό εργατικό ατύχημα – Αυτοκίν...,2025-05-06,07:47:44,Κοινωνία,,Σοβαρό εργατικό ατύχημα σημειώθηκε το πρωί της...,"Σύμφωνα με την ΕΡΤ, αυτοκίνητο που ερχόταν από...",https://www.naftemporiki.gr/wp-content/themes/...,Τραυματίστηκε ένας εργάτης- Τι συνέβη
2,https://www.naftemporiki.gr/society/1934740/ir...,Ηράκλειο: Εργατικό ατύχημα σε επιχείρηση με μά...,2025-03-19,10:44:47,Κοινωνία,,Εργατικό ατύχημα σημειώθηκε σε επιχείρηση με μ...,"Όπως έγινε γνωστό, το περιστατικό σημειώθηκε χ...",https://www.naftemporiki.gr/wp-content/themes/...,Συνελήφθη ο ιδιοκτήτης της επιχείρησης
3,https://www.naftemporiki.gr/society/1806579/ko...,Κοζάνη: Εργατικό ατύχημα με 22χρονο σοβαρά τρα...,2024-10-24,18:01:10,Κοινωνία,,Εργατικό ατύχημα σημειώθηκε σήμερα στην Κοζάνη...,"Συγκεκριμένα, ο 22χρονος εργαζόμενος σε εργολη...",https://www.naftemporiki.gr/wp-content/themes/...,"Σύμφωνα με τον διοικητή του Μποδοσάκειου, η κα..."
4,https://www.naftemporiki.gr/culture/ekdiloseis...,ΠΟΘΑ: Στάση εργασίας και συγκέντρωση διαμαρτυρ...,2024-10-16,13:08:24,Πολιτιστικά \Εκδηλώσεις,,Η Πανελλήνια Ομοσπονδία Θεάματος Ακροάματος (Π...,"Σύμφωνα με την ΠΟΘΑ, ο τεχνικός έπεσε από ύψος...",https://www.naftemporiki.gr/wp-content/themes/...,Συγκέντρωση έξω από το υπουργείο Πολιτισμού αύ...
5,https://www.naftemporiki.gr/society/1746755/gs...,ΓΣΕΕ: Νέο εργατικό ατύχημα στο λιμάνι του Πειραιά,2024-08-22,15:15:40,Κοινωνία,,"«Ένας ακόμα εργαζόμενος, αυτή τη φορά στο λιμά...",Όπως επισημαίνει η Συνομοσπονδία σε ανακοίνωσή...,https://www.naftemporiki.gr/wp-content/themes/...,Η Συνομοσπονδία ζητά την άμεση παρέμβαση των α...
6,https://www.naftemporiki.gr/maritime/1746639/l...,Εργατικό ατύχημα στο Λιμάνι Πειραιά: Έπεσε πάν...,2024-08-22,13:23:30,Ναυτιλία,,Την πιστή εφαρμογή των κανόνων Υγείας και Ασφά...,Σύμφωνα με το υπουργείο Ναυτιλίας ο τραυματίας...,https://www.naftemporiki.gr/wp-content/themes/...,Ο τραυματίας έχει μεταφερθεί με ασθενοφόρο όχη...
7,https://www.naftemporiki.gr/society/1634672/pi...,Πήλιο: Εργατικό ατύχημα με εργαζόμενη στην καθ...,2024-04-08,11:55:29,Κοινωνία,,,"Eργατικό ατύχημα υπέστη εργαζόμενη, συνοδός απ...",https://www.naftemporiki.gr/wp-content/themes/...,"Έσπασε το χέρι της, το οποίο μπλέχτηκε στον κά..."
8,https://www.naftemporiki.gr/stories/1576297/i-...,Η πρώτη συμφωνία αποζημίωσης για εργατικό ατύχ...,2024-01-26,06:00:46,Stories,,,Το ημερολόγιο γράφει 26 Ιανουαρίου 1695 όταν σ...,https://www.naftemporiki.gr/wp-content/themes/...,Η ιστορία του θρυλικού Γουίλιαμ Κιντ - Σαν σήμ...
9,https://www.naftemporiki.gr/kosmos/1533951/not...,Νότια Κορέα: Εργατικό ατύχημα ο θάνατος αεροσυ...,2023-11-07,15:01:10,Κόσμος,,,Έπειτα από μια ιστορική απόφαση σύμφωνα με την...,https://www.naftemporiki.gr/wp-content/themes/...,"Η Korean Air ανακοίνωσε ότι ""τηρεί αυστηρά"" τ..."


In [ ]:
articles_df.to_csv("/content/drive/MyDrive/CSVFILES/naftemporiki_articles.csv", index=False)